In [2]:
import pandas as pd
from lxml import html
import numpy as np
from lxml import etree
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, f1_score
from collections import Counter, defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

C:\Users\1\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
def get_embedding(text, model, dim, tfidf=False):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            if tfidf:
                vectors[i] = (v * (words[word] / total)) * tfidf[word]
            else:
                vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

# Немного парсинга

In [4]:
corpus = html.fromstring(open('paraphrases.xml', 'rb').read())

In [5]:
text1 =[]
text2=[]
label =[]

<paraphrase>
      <value name="id">1</value>
      <value name="id_1">201</value>
      <value name="id_2">8159</value>
      <value name="text_1">Полицейским разрешат стрелять на поражение по гражданам с травматикой.</value>
      <value name="text_2">Полиции могут разрешить стрелять по хулиганам с травматикой.</value>
      <value name="jaccard">0.65</value>
      <value name="class">0</value>
</paraphrase>

In [6]:
for phrase in corpus.xpath('//paraphrase'):
    text1.append(phrase.xpath('./value[@name="text_1"]/text()')[0])
    text2.append(phrase.xpath('./value[@name="text_2"]/text()')[0])
    label.append(phrase.xpath('./value[@name="class"]/text()')[0])

In [7]:
data = pd.DataFrame({'text_1':text1, 'text_2':text1, 'label':label})

In [8]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [9]:
data['text_1_token'] = data['text_1'].apply(tokenize)
data['text_2_token'] = data['text_2'].apply(tokenize)

In [10]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_token,text_2_token
0,0,Полицейским разрешат стрелять на поражение по ...,Полицейским разрешат стрелять на поражение по ...,полицейский разрешить стрелять поражение гражд...,полицейский разрешить стрелять поражение гражд...,полицейским разрешат стрелять на поражение по ...,полицейским разрешат стрелять на поражение по ...
1,0,Право полицейских на проникновение в жилище ре...,Право полицейских на проникновение в жилище ре...,право полицейский проникновение жилища решить ...,право полицейский проникновение жилища решить ...,право полицейских на проникновение в жилище ре...,право полицейских на проникновение в жилище ре...
2,0,Президент Египта ввел чрезвычайное положение в...,Президент Египта ввел чрезвычайное положение в...,президент египет ввести чрезвычайный положение...,президент египет ввести чрезвычайный положение...,президент египта ввел чрезвычайное положение в...,президент египта ввел чрезвычайное положение в...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Вернувшихся из Сирии россиян волнует вопрос тр...,вернуться сирия россиянин волновать вопрос тру...,вернуться сирия россиянин волновать вопрос тру...,вернувшихся из сирии россиян волнует вопрос тр...,вернувшихся из сирии россиян волнует вопрос тр...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,В Москву из Сирии вернулись 2 самолета МЧС с р...,москва сирия вернуться 2 самолёт мчс россиянин...,москва сирия вернуться 2 самолёт мчс россиянин...,в москву из сирии вернулись 2 самолета мчс с р...,в москву из сирии вернулись 2 самолета мчс с р...


## Данные для обучения

In [11]:
data_learn = pd.read_csv('news_texts.csv')

In [12]:
data_learn = data_learn.dropna()

In [13]:
data_learn.head()

,content,content_norm
0,Канцлер Германии Ангела Меркель в ходе брифинг...,канцлер германия ангел меркель ход брифинг пре...
1,Российские и белорусские войска успешно заверш...,российский белорусский войско успешно завершит...
2,"Дзюба, Шатов и Анюков оказались не нужны «Зени...",дзюба шат анюк оказаться нужный зенит российск...
3,"В Испанию без фанатов\nПожалуй, главной пятнич...",испания фанат пожалуй главный пятничный новост...
4,"Постпред России при ООН Виталий Чуркин, говоря...",постпред россия оон виталий чуркин говорить ве...


## Вектора

Преобразуйте тексты в векторы в каждой паре 4 методами  - SVD, NMF, Word2Vec, Fastext. Для SVD и NMF сделайте две пары векторов - через TfidfVectorizer и CountVectorizer. Для word2vec сделайте две пары векторов - с взвешиванием по tfidf и без. Для Fastext постройте две модели - без нормализации и с нормализацией, а через каждую модель постройте две пары векторов -  с взвешиванием по tfidf и без. 

In [14]:
y = data['label'].values

In [15]:
c = CountVectorizer(max_features=10000) #5000
cv = c.fit(data_learn['content_norm']) 

In [16]:
t = TfidfVectorizer(max_features=10000)#5000
tv = t.fit(data_learn['content_norm'])

**SVD

In [17]:
m_svd_cv = TruncatedSVD(300) #150
m_svd_tv = TruncatedSVD(300) #150

In [19]:
m_svd_cv.fit(cv)
m_svd_tv.fit(tv)

In [20]:
#CountVectoriser
a = m_svd_cv.transform(cv.transform(data['text_1_norm']))
b = m_svd_cv.transform(cv.transform(data['text_2_norm']))
svd_cv = [a, b]

In [101]:
#TFIDF
a = m_svd_tv.transform(tv.transform(data['text_1_norm']))
b = m_svd_tv.transform(tv.transform(data['text_2_norm']))
svd_tv = [a, b]

C:\Users\1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


**NMF

In [21]:
m_nmf_cv = NMF(300) #150
m_nmf_tv = NMF(300)

In [23]:
#CountVectoriser
a = m_nmf_cv.fit_transform(cv.transform(data['text_1_norm']))
b = m_nmf_cv.fit_transform(cv.transform(data['text_2_norm']))
nmf_cv = [a, b]

In [24]:
#TFIDF
a = m_nmf_tv.fit_transform(tv.transform(data['text_1_norm']))
b = m_nmf_tv.fit_transform(tv.transform(data['text_2_norm']))
nmf_tv = [a, b]

**Word2Vec

In [112]:
w2v = gensim.models.Word2Vec([text.split() for text in data_learn['content_norm'].values], size=300, sg=1) #150

In [118]:
#без TFIDF
dim = 300 #150
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [114]:
w2v_notv = [X_text_1_w2v, X_text_1_w2v]

In [129]:
#c TFIDF
t = TfidfVectorizer(max_features=5000)
tv_dop = t.fit_transform(data_learn['content_norm'])

dim = 300 #150
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

# t1 = t.fit(X_text_1_w2v).vocabulary_
# t2 = t.fit(X_text_2_w2v).vocabulary_

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, tv_dop)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, tv_dop)

C:\Users\1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [130]:
w2v_tv = [X_text_1_w2v, X_text_1_w2v]

**FastText

In [135]:
ft_norm = gensim.models.FastText([text.split() for text in data_learn['content_norm'].values], 
                              size=150, min_n=4, max_n=8)
ft = gensim.models.FastText([text.split() for text in data_learn['content'].values], 
                              size=150, min_n=4, max_n=8)

In [136]:
#norm, TFIDF
dim = 300 #150
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, tv_dop)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, tv_dop)
    
ft_n_tv = [X_text_1_ft, X_text_2_ft]

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [137]:
#norm, no TFIDF
dim = 300 #150
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)
    
ft_n = [X_text_1_ft, X_text_2_ft]

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [138]:
#not norm, TFIDF
dim = 300 #150
X_text_1_ft = np.zeros((len(data['text_1_token']), dim))
X_text_2_ft = np.zeros((len(data['text_2_token']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, tv_dop)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, tv_dop)
    
ft_tv = [X_text_1_ft, X_text_2_ft]

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [139]:
#not norm, no TFIDF
dim = 300 #150
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)
    
ft = [X_text_1_ft, X_text_2_ft]

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


## Косинусная близость

In [164]:
def cos_dist(vecs):
    vec_1 = vecs[0]
    vec_2 = vecs[1]
    result = []
    
    for i in range(len(vec_1)):
        cos_dis = cosine_distances(vec_1[i].reshape(1, -1), vec_2[i].reshape(1, -1))
        result.append(cos_dis[0][0])
    return result

In [167]:
%%time
res = cos_dist(svd_tv)
results = pd.DataFrame({'svd_tv':res})
res = cos_dist(svd_cv)
results['svd_cv'] = res

res = cos_dist(nmf_tv)
results['nmf_cv'] = res
res = cos_dist(nmf_cv)
results['nmf_cv'] = res

res = cos_dist(w2v_tv)
results['w2v_tv'] = res
res = cos_dist(w2v_notv)
results['w2v_notv'] = res

res = cos_dist(ft_n_tv)
results['ft_n_tv'] = res
res = cos_dist(ft_n)
results['ft_n'] = res
res = cos_dist(ft_tv)
results['ft_tv'] = res
res = cos_dist(ft)
results['ft'] = res

Wall time: 10.1 s


In [168]:
results.head()

,svd_tv,svd_cv,nmf_cv,w2v_tv,w2v_notv,ft_n_tv,ft_n,ft_tv,ft
0,0.000000e+00,2.220446e-16,0.790547,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0
1,2.220446e-16,3.330669e-16,0.054093,2.220446e-16,2.220446e-16,1.0,1.0,1.0,1.0
2,0.000000e+00,0.000000e+00,0.480549,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0
3,0.000000e+00,0.000000e+00,0.006462,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0
4,0.000000e+00,4.440892e-16,0.003660,3.330669e-16,3.330669e-16,1.0,1.0,1.0,1.0


## Обучение модели

In [173]:
X_train, X_test, y_trai, y_test = train_test_split(results, y, random_state=1)

In [177]:
from sklearn.model_selection import cross_val_score

In [179]:
clf = LogisticRegression(C=100, class_weight='balanced', max_iter=1500, n_jobs=-1)
cross_val_score(clf, results, y, cv=5).mean()

C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect whe

0.4047373954157553

## Перебор параметров

Я попыталсь увеличить некоторые параметры. Но что-то пошло не так. Нули в FastText были и изначально, я так и не смогла понять, что не так.